In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
'''
Funcao Sigmoide
Aplica a funcao de sigmoide em todos os elementos do vetor
Parametros: x - vetor. EX: [1.1,2.2]
Retorno - retorna o vetor com todos os elementos aplicados a funcao sigmoide
'''
def sigmoide(x):
    
    S = [1/(1 + math.exp(-x_i)) for x_i in x]

    return S
   

In [3]:

'''
Funcao Derivada Sigmoide para um vetor
Recebe um vetor e aplica a sigmoide em todos os elementos do vetor
parametro: x - vetor. Ex: [1,2.5,3]
Retorno: O vetor com todos os elementos aplicados a funcao sigmoide
'''
def derivada_sigmoide(x):
    S = [(1-x)*x for x_i in x]
    return S


In [4]:
'''
Funcao Gradiente
Calcula o gradiente da matriz dados, de acordo com a matriz de peso
Parametros:
dados - Matriz de dados
matriz_peso_a - Matriz de Pesos A
matriz_peso_b - Matriz de Pesos B
saida - matriz de saida
Retorno:
Retorna a derivada de j em relacação a matriz A e a derivada de j em relação a matriz B
'''
def gradiente(dados, matriz_peso_a,matriz_peso_b, saida):
    matriz_y,matriz_z = calcula_saida(dados, matriz_peso_a,matriz_peso_b,len(dados),len(dados[0]))
    erro = np.array(matriz_y) - np.array(saida)
    
    aux = np.empty((len(erro),len(erro[0])))
    
    
    for i in range(0, len(erro)):
        for j in range(0, len(erro[0])):
            aux[i][j] = erro[i][j] #* derivada_sigmoide_para_x(matriz_y[i][j])
            
    derivada_j_em_b = np.dot(aux.T,matriz_z)
  
    
    
    derivada_j_em_sigma = np.dot(aux,matriz_peso_b)#somatorio
    
    matriz_z *= 1-matriz_z
   
    aux2 = derivada_j_em_sigma * matriz_z
    derivada_j_em_a = np.dot(aux2.T,dados)

    
    
    return derivada_j_em_a[:-1], derivada_j_em_b


In [5]:
		
'''
Funcão Calcula Saida
Parametros:
dados - Conjunto de dados. EX: [[1.0,1],[1,2]]
matriz_peso_a - Matriz de pesos da camada principal
matriz_peso_b - Matriz de pesos da camada oculta
dimensao_linha_dados - tamanho do conjunto de entradas. EX: 5 (qnt de linhas)
dimensao_coluna_dados - quantidade de dimensoes. EX: 5 (qnt de colunas)
Retorno: Matriz com as saidas
'''
def calcula_saida(dados, matriz_peso_a, matriz_peso_b, dimensao_linha_dados, dimensao_coluna_dados):
    dados = np.array(dados)
    
    matriz_z_in = np.dot(dados,matriz_peso_a.T) #dados x matriz transposta de pesos

    matriz_z = np.zeros((dimensao_linha_dados,len(matriz_peso_a)))
    for i in range(0, len(matriz_z_in)):

        matriz_z[i] = sigmoide(matriz_z_in[i])
        
    b = np.ones((len(matriz_z_in),1))
    matriz_z = np.hstack((matriz_z,b))#Adiciona Bias em Z
    
    matriz_y_in = np.dot(matriz_z,matriz_peso_b.T) #Operação da Segunda camada
    
    
    matriz_y = np.zeros((dimensao_linha_dados,len(matriz_peso_b)))

    for i in range(0, len(matriz_y_in)):
        matriz_y[i] = matriz_y_in[i]#sigmoide(matriz_y_in[i])

    
    return matriz_y,matriz_z



In [6]:
'''
Funcao rede_neural
Parametros:
dados - Matriz de dados. EX: [[1,2,312,3.21,3],[1,2,3,4.4]]
saidas - Matriz de classes. EX[[1,0,0],[0,1,0],[0,0,1]]
qnt_neuronios - Quantida de neuronios na  primeira camada
matriz_peso_a - Matriz de Pesos na primeira camada, caso não passe, ela é gerada!
matriz_peso_b - Matriz de Pesos na segunda camada, caso não passe, ela é gerada!
erro_minimo - Erro minimo considerado para que a rede esteja treinada
epocas - quantidade maxima de epocas que a rede continuará treinando
'''
def rede_neural_mlp_back_propagation(dados_validacao, saida_validacao, dados, saidas, qnt_neuronios, matriz_peso_a = [], matriz_peso_b = [], erro_minimo=0.00005,epocas = 5000):
    for i in dados:
        i.append(1)
    for i in dados_validacao:
        i.append(1)
        
    dimensao_coluna_dados = len(dados[0])
    dimensao_linha_dados = len(dados)
    dimensao_coluna_saidas = len(saidas[0])

    if matriz_peso_a == []:
        for i in range(0, qnt_neuronios): #Linhas
            aux = []
            for j in range(0, dimensao_coluna_dados):#Colunas

                aux.append(random.uniform(-0.2,0.2))
            matriz_peso_a.append(aux)

    #Matriz B de pesos
    if matriz_peso_b == []:
        for i in range(0, len(saidas[0])):#Linhas
            aux = []
            for j in range(0, qnt_neuronios+1):
                aux.append(random.uniform(-0.2,0.2))
            matriz_peso_b.append(aux)

    matriz_peso_a = np.array(matriz_peso_a)
    matriz_peso_b = np.array(matriz_peso_b)
    
    saida_da_rede,a = calcula_saida(dados, matriz_peso_a,matriz_peso_b,dimensao_linha_dados,dimensao_coluna_dados)
    
    saidas = np.array(saidas)
    saida_validacao = np.array(saida_validacao)
    
    
    
    erro = saida_da_rede - saidas #Subtração das matrizes
    erro_quadratico_medio = np.sum(erro**2)/len(erro) #Todos os elementos ao quadrado, depois soma todos os elementos, divido pela qnt de elementos (linhas e colunas)
    
    numero_epocas = 0
    matriz_erro = [[numero_epocas,erro_quadratico_medio]]
    
    saida_rede_validacao,b = calcula_saida(dados_validacao, matriz_peso_a,matriz_peso_b,len(dados_validacao),len(dados_validacao[0]))
        

    erro_validacao = saida_rede_validacao - saida_validacao
    erro_quadratico_medio_validacao = np.sum(erro_validacao**2)/len(erro_validacao)
    
    matriz_peso_ideal_a = matriz_peso_a
    matriz_peso_ideal_b = matriz_peso_b
    
    erro_quadratico_medio_validacao_atual = erro_quadratico_medio_validacao
    matriz_erro_validacao = [[numero_epocas,erro_quadratico_medio_validacao_atual]]
    #BackPropagation
    alpha = 1
    
    
    #while erro_quadratico_medio > erro_minimo and numero_epocas < epocas:
    cont = 0
    while numero_epocas < epocas and erro_quadratico_medio > erro_minimo and cont < 20:
        
        numero_epocas +=1
        derivada_j_em_a, derivada_j_em_b = gradiente(dados, matriz_peso_a,matriz_peso_b, saidas)
               
        
        derivada_j_em_a_aux = derivada_j_em_a * alpha
        derivada_j_em_b_aux = derivada_j_em_b * alpha
        

        
        matriz_peso_a_aux = matriz_peso_a - derivada_j_em_a_aux
        matriz_peso_b_aux = matriz_peso_b - derivada_j_em_b_aux
        
        matriz_y,b = calcula_saida(dados, matriz_peso_a_aux,matriz_peso_b_aux,dimensao_linha_dados,dimensao_coluna_dados)

        erro = matriz_y - saidas #Subtração das matrizes
        
        
        erro_quadratico_medio_atual = np.sum(erro**2)/len(erro) #Todos os elementos ao quadrado, depois soma todos os elementos, divido pela qnt de elementos (linhas e colunas)

                
        while erro_quadratico_medio_atual > erro_quadratico_medio:
                       
            alpha = alpha/2.0
            
            
            derivada_j_em_a_aux = derivada_j_em_a * alpha
            derivada_j_em_b_aux = derivada_j_em_b * alpha
            

            matriz_peso_a_aux = matriz_peso_a - derivada_j_em_a_aux
            matriz_peso_b_aux = matriz_peso_b - derivada_j_em_b_aux
            matriz_y,b = calcula_saida(dados, matriz_peso_a_aux,matriz_peso_b_aux,dimensao_linha_dados,dimensao_coluna_dados)

            erro = matriz_y - saidas #Subtração das matrizes
            
            erro_quadratico_medio_atual = np.sum(erro**2)/len(erro) #Todos os elementos ao quadrado, depois soma todos os elementos, divido pela qnt de elementos (linhas e colunas)
            
            

           
        alpha = alpha*2

        
        erro_quadratico_medio = erro_quadratico_medio_atual
        matriz_erro.append([numero_epocas,erro_quadratico_medio])
        matriz_peso_a = np.array(matriz_peso_a_aux)
        matriz_peso_b = np.array(matriz_peso_b_aux)


        saida_rede_validacao,b = calcula_saida(dados_validacao, matriz_peso_a,matriz_peso_b,len(dados_validacao),len(dados_validacao[0]))
        erro_quadratico_medio_validacao = erro_quadratico_medio_validacao_atual
        erro_validacao = saida_rede_validacao - saida_validacao
        erro_quadratico_medio_validacao_atual = np.sum(erro_validacao**2)/len(erro_validacao)
        matriz_erro_validacao.append([numero_epocas,erro_quadratico_medio_validacao_atual])
        
        print(erro_quadratico_medio_validacao_atual)
        
        if erro_quadratico_medio_validacao_atual > erro_quadratico_medio_validacao:
            cont +=1
        else:
            matriz_peso_ideal_a = matriz_peso_a
            matriz_peso_ideal_b = matriz_peso_b
            cont = 0
        
    return matriz_peso_ideal_a,matriz_peso_ideal_b,matriz_erro,matriz_erro_validacao

In [7]:
'''
Funcao rede_neural
Parametros:
dados - Matriz de dados. EX: [[1,2,312,3.21,3],[1,2,3,4.4]]
saidas - Matriz de classes. EX[[1,0,0],[0,1,0],[0,0,1]]
qnt_neuronios - Quantida de neuronios na  primeira camada
matriz_peso_a - Matriz de Pesos na primeira camada, caso não passe, ela é gerada!
matriz_peso_b - Matriz de Pesos na segunda camada, caso não passe, ela é gerada!
erro_minimo - Erro minimo considerado para que a rede esteja treinada
epocas - quantidade maxima de epocas que a rede continuará treinando
'''
matriz_pesos_a, matriz_pesos_b,matriz_erro,matriz_erro_validacao = rede_neural_mlp_back_propagation(dados_validacao =[[0.1,0.1],[0.15,1.05],[1.08,0.01],[1.1,0.95]], saida_validacao =[[0],[1],[1],[0]],dados = [[0,0],[0,1],[1,0],[1,1]], saidas = [[0],[1],[1],[0]], qnt_neuronios = 2, matriz_peso_a = [], matriz_peso_b = [], erro_minimo=0.00005,epocas = 10000)

0.3118692598302586
0.2683291285117001
0.25522014651793407
0.2514371458185262
0.25041183086728164
0.25008500732711886
0.25001141715677844
0.2499799891908947
0.24997668025294367
0.249972768172041
0.24997327289802548
0.24997259041285808
0.2499728719282226
0.24997273441231885
0.24997298357107559
0.24997282792787218
0.2499730967735905
0.24997292094800289
0.24997321180158247
0.24997301347465722
0.2499733289666317
0.24997310551611462
0.24997344863465634
0.24997319708881155
0.24997335202590637
0.24997328821567727
0.24997345372128227
0.24997337894017135
0.24997355577260444
0.24997346931880848
0.24997365825447637
0.2499735594287858
0.24997376125282816
0.24997364937303473
0.2499738648668366
0.24997373928638142
0.24997396921120163
0.2499738698047797
0.2499740744232326
0.24997396140412836
0.2499741806543943
0.24997405253411575
0.24997428808469746
0.2499741431913356
0.24997439692405715
0.24997423337601055
0.2499745074177362
0.24997432309309692
0.2499746198528356
0.24997441235364742
0.249974547452683

0.2499914585535879
0.24999144458389677
0.2499914599160763
0.24999144424476138
0.24999146138463552
0.24999144378571547
0.2499914630012285
0.24999144321310385
0.24999144957076969
0.24999144253431893
0.2499914493381223
0.24999144176003768
0.24999144902706805
0.24999144090298925
0.24999144864546177
0.2499914399791052
0.2499914482024812
0.2499914390082576
0.2499914477088794
0.24999143801516183
0.24999144717728883
0.24999143703047857
0.2499914466225866
0.24999143657896866
0.2499914460618779
0.2499914348718944
0.24999144551626845
0.24999143303574806
0.24999144501036238
0.24999143107337946
0.24999144457338618
0.2499914289885139
0.24999144424012104
0.24999142678597253
0.24999144405203044
0.24999142447194295
0.24999143000030322
0.24999142205366964
0.24999142801869922
0.24999141954165324
0.2499914259607554
0.24999141694846488
0.24999142383428358
0.2499914142898927
0.24999142164843258
0.249991411585694
0.24999141941394643
0.24999140886051602
0.24999141714347586
0.24999140614502335
0.24999141485195

0.24998842544525335
0.2499884398719195
0.24998839704357362
0.24998839765346653
0.24998836858364304
0.24998837003254615
0.24998834010826113
0.24998834240952145
0.24998831167221952
0.24998831481448397
0.24998828334746234
0.24998828728442762
0.2499882552280059
0.2499882598649394
0.24998822743623036
0.24998823261232495
0.2499882052778186
0.24998820559504836
0.24998817552304892
0.24998817890034059
0.24998814562495106
0.2499881526355571
0.24998811559822476
0.2499881269344204
0.24998808546315035
0.24998810196381122
0.24998805524729753
0.24998805604769617
0.24998802498581305
0.24998802669046832
0.24998799472940036
0.24998799734621038
0.24998796454318298
0.24998796805003395
0.24998793451283619
0.24998793884525305
0.24998790475055452
0.2499879097854274
0.2499878825398909
0.2499878809358163
0.2499878510436321
0.24998785237956012
0.24998781939081233
0.2499878242191899
0.24998778759446777
0.24998779658261633
0.2499877556728765
0.24998776962973174
0.2499877236512021
0.24998772389586454
0.24998769156

0.24997744942711014
0.24997735125577414
0.24997736861622355
0.24997726753429303
0.2499772887333999
0.24997720805566287
0.24997721017372093
0.24997711904146053
0.2499771334657055
0.249977029556114
0.24997705930511888
0.24997693969161466
0.2499769886158323
0.24997684958926059
0.24997685560248292
0.24997675945371406
0.2499767697469037
0.2499766695982304
0.24997668434047962
0.24997658046928886
0.24997659964371222
0.24997649270596117
0.24997651599847648
0.24997642970608064
0.24997643385048698
0.2499763360989407
0.24997635379805883
0.24997624199901988
0.24997627663195937
0.24997614751306368
0.24997620340631774
0.24997605280526133
0.24997606025425345
0.24997595811418036
0.24997597021895343
0.24997586380522313
0.24997588072736382
0.2499757704009707
0.24997579208058396
0.24997567865163875
0.24997570467539026
0.2499756110306477
0.24997561903132148
0.2499755125246197
0.24997553584861198
0.24997541351573982
0.2499754560568313
0.24997531413441998
0.24997538090057228
0.24997521458084992
0.2499752242

0.24994028045292527
0.24993986301956328
0.2499399877135303
0.24993948237653044
0.24993972209914836
0.24993909960247884
0.2499391593993449
0.24993871700239323
0.2499388108609194
0.24993833867472334
0.24993846947881032
0.24993797151617442
0.24993813959833408
0.24993771278411975
0.24993782757383765
0.24993731125308222
0.24993754291862874
0.24993690712774366
0.24993696591016482
0.24993650270808745
0.2499365975536436
0.24993610213869077
0.24993623648429336
0.24993571246952628
0.24993588724672133
0.2499354444776249
0.24993555651197027
0.2499350205597075
0.249935254305855
0.24993459368453882
0.24993499566916066
0.24993416629485601
0.2499342650672671
0.24993374269014665
0.2499338838635496
0.24993333041578142
0.24993351528339358
0.24993304920271206
0.2499331665537493
0.24993260122924585
0.24993284858574474
0.24993215004855043
0.24993257783095607
0.24993169833561102
0.24993180454343514
0.24993125083601836
0.249931402803666
0.2499308159302278
0.2499310150411891
0.249930517132245
0.249930649338419

0.24891339368467696
0.24886080735033195
0.2488722116047885
0.24883681448321326
0.2488584578043566
0.24881219806097665
0.248821064986564
0.24878366895270798
0.2488027629925169
0.24876104519148112
0.248799474644257
0.24872892230430071
0.24874549574040225
0.2487069648586823
0.248735393644919
0.24867173179346208
0.24868592789410132
0.2486496085207805
0.24867025642279744
0.24861151263481784
0.2486235389454409
0.24857729285983443
0.24860315051661597
0.2485477893709731
0.2485578885280758
0.24850921400105877
0.24853331348097046
0.2484801463521096
0.24852944101748659
0.2484376317069397
0.24846023358936412
0.24840820549977186
0.24845065635286478
0.24836190044045375
0.2483832909500317
0.24833155739222668
0.2483689009214372
0.24828149134149713
0.24830201073265123
0.24824979785395235
0.2482835123141217
0.2481959007000644
0.24821594010259296
0.248151007192854
0.248193987708435
0.24810462702371633
0.24812465831437253
0.24805521741398828
0.248099803972083
0.2480071564462388
0.24802770886907377
0.24795